In [1]:
#imports
import websockets
import asyncio
import time
import json
import re
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
load_dotenv()
import os

In [2]:
#alpaca url 
base_url = 'https://paper-api.alpaca.markets'
#alpaca api keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
#alpaca object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url
)

In [ ]:
#Function to gather live feed and trade CPI
"""
Here we use a websocket instead of the twitter API, because the twitter API has a few second delay. The websocket
allows for a constant connection between us and the treeofalpha websocket. Regex is then used to seperate the CPI
data from the news break and again a trade is made depending on the data vs expectations.

"""



async def listen():
    #url we gather cpi data from
    url = 'wss://news.treeofalpha.com/ws'

    #async continually pings the websocket asking to receive new messages 
    async with websockets.connect(url) as websocket:
        while True:
            msg = await websocket.recv()
            data = json.loads(msg)
            #regular expression used to clean CPI numbers
            matches = re.match(r"U\.S\.\s+CPI:\s+\+((\d+)(?:\.\d+)?)%.+?\nU\.S\.\s+CORE\s+CPI:\s+\+((\d+)(?:\.\d+)?)%.+", data['title'])
            t0 = time.time()

            cpi = int(float(matches.group(1)))
            core_cpi = int(float(matches.group(3)))

            #conditionals to determine short/long
            if cpi >= 6.5:
                alpaca_api.submit_order(
                symbol='SPY',
                qty=100,  
                side='sell',
                type='limit',
                limit_price= 375,
                time_in_force='day',
                extended_hours=True,
            )
                t1 = time.time()
                print(t1-t0)
                break 
            elif cpi <= 6.4:
                alpaca_api.submit_order(
                symbol='BTCUSD',
                qty=15,  
                side='buy',
                type='market',
                time_in_force='gtc',
            )
                t1 = time.time()
                print(t1-t0)
                break
            else:
                print("No trade to be made given current data.")
                t1 = time.time()
                print(t1-t0)
                break

In [3]:
#Show how getting the data works, also showing how to ping a request

import websocket
import time
import json
import re

"""
Example of how the code gets the CPI data from the news break
"""

def fetch():
    endpoint = 'wss://news.treeofalpha.com/ws'
    ws = websocket.WebSocket()
    ws.connect(endpoint)
    start = time.time()
    ws.send('cpi')
    message = ws.recv()
    data = json.loads(message)
    print(data)
    matches = re.match(r"U\.S\.\s+CPI:\s+\+((\d+)(?:\.\d+)?)%.+?\nU\.S\.\s+CORE\s+CPI:\s+\+((\d+)(?:\.\d+)?)%.+", data['title'])
    print('\n')
    print("CPI: " + matches.group(1))
    print('\n')
    print("Core CPI: " + matches.group(3))
    print('\n')
    print(time.time() - start)

fetch()

{'title': 'U.S. CPI: +8.4% YEAR-OVER-YEAR (EST. +8.1%)\nU.S. CORE CPI: +6.5% YEAR-OVER-YEAR (EST. +6.3%)', 'source': 'Terminal', 'url': 'https://news.treeofalpha.com/?v=1683567997266', 'time': 1683567997266, 'symbols': []}


CPI: 8.4


Core CPI: 6.5


0.09516310691833496
